<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Train_35.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers
from datasets import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Download Data

In [2]:
%cd "/content/Dataset(s)/text-image-ferramenta/"

/content/Dataset(s)/text-image-ferramenta


In [ ]:
# !gdown https://drive.google.com/uc?id=16whYrlXvaWMgMrn1yCqv1EmFSW-PykNH

In [ ]:
# !gdown https://drive.google.com/uc?id=179e1TGYPti49AKHG8v5JZ9yMaNJgQwyZ

In [ ]:
# !gdown https://drive.google.com/uc?id=1ILTZBqbZ8g5pnuSBSd0uCsra0lEnJYRb

In [ ]:
# !gdown https://drive.google.com/uc?id=1KlN-nBCyraJsfYLyFdUQQbZMZWFNG-S1

Extract Data

In [ ]:
import tarfile

with tarfile.open('./text-val.tar.gz', 'r:gz') as tar_ref:
    tar_ref.extractall('./ferramenta_texts/')

with tarfile.open('./text-train.tar.gz', 'r:gz') as tar_ref:
    tar_ref.extractall('./ferramenta_texts')

with tarfile.open('./images-val.tar.gz', 'r:gz') as tar_ref:
    tar_ref.extractall('./ferramenta_Images/')

with tarfile.open('./images-train.tar.gz', 'r:gz') as tar_ref:
    tar_ref.extractall('./ferramenta_Images/')

Create Label Files

In [ ]:
#Create txt file for Image Paths and Labels
import os

dataset_folder = './ferramenta_Images/images-train'
output_file = './ferramenta_Images/train_dataset.txt'

def custom_sort_key(item):
    return item.replace('@', '')

with open(output_file, 'w') as f:
    for label in sorted(os.listdir(dataset_folder), key=custom_sort_key):
        label_path = os.path.join(dataset_folder, label)
        if os.path.isdir(label_path):
            class_name = label
            image_files = sorted(os.listdir(label_path))
            for image_file in image_files:
                image_path = os.path.join(label_path, image_file)
                f.write(f'{image_path}, {class_name}\n')

In [ ]:
#Create txt file for Image Paths and Labels
import os

dataset_folder = './ferramenta_Images/images-val'
output_file = './ferramenta_Images/test_dataset.txt'

def custom_sort_key(item):
    return item.replace('@', '')

with open(output_file, 'w') as f:
    for label in sorted(os.listdir(dataset_folder), key=custom_sort_key):
        label_path = os.path.join(dataset_folder, label)
        if os.path.isdir(label_path):
            class_name = label
            image_files = sorted(os.listdir(label_path))
            for image_file in image_files:
                image_path = os.path.join(label_path, image_file)
                f.write(f'{image_path}, {class_name}\n')

Load Image Data From Label Files

In [3]:
import os

image_file_paths = []
image_genre_labels = []

labels_file = "./ferramenta_Images/train_dataset.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()
        image_file_paths.append(filename)
        image_genre_labels.append(labels)

In [4]:
print(len(image_file_paths))
print(len(image_genre_labels))

66141
66141


In [5]:
import os

test_image_file_paths = []
test_image_genre_labels = []

labels_file = "./ferramenta_Images/test_dataset.txt"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()
        test_image_file_paths.append(filename)
        test_image_genre_labels.append(labels)

In [6]:
print(len(test_image_file_paths))
print(len(test_image_genre_labels))

21869
21869


Load Text Data From CSV

In [7]:
import pandas as pd
df = pd.read_csv("./ferramenta_texts/text-train.csv", index_col='image')
df_test = pd.read_csv("./ferramenta_texts/text-val.csv", index_col='image')

In [ ]:
df.head()

In [8]:
df['text'] = df["title"] + df['description']
df_test['text'] = df_test["title"] + df_test['description']

In [9]:
# Remove all the numeric data from text ! Why ? i dont know, feels right to do so.
import re

def remove_numbers(text):
    if isinstance(text, str):
        return re.sub(r'\d+', '', text)
    else:
        return text

In [10]:
df['text'] = df['text'].apply(remove_numbers)
df_test['text'] = df_test['text'].apply(remove_numbers)

In [11]:
df['text'].fillna("no text available", inplace=True)
df_test['text'].fillna("no text available", inplace=True)

In [12]:
texts_csv_lst = df['text'].tolist()
test_texts_csv_lst = df_test['text'].tolist()

In [13]:
len(texts_csv_lst) , len(test_texts_csv_lst)

(66141, 21869)

In [ ]:
# texts_csv_lst[1000]

Label Encoder

In [14]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(image_genre_labels)

LabelEncoder()

In [15]:
transformed_genre_labels = label_encoder.transform(image_genre_labels)
test_transformed_genre_labels = label_encoder.transform(test_image_genre_labels)

In [16]:
transformed_genre_labels = transformed_genre_labels.tolist()
test_transformed_genre_labels = test_transformed_genre_labels.tolist()

In [17]:
len(transformed_genre_labels), len(test_transformed_genre_labels)

(66141, 21869)

Create Dataset

In [ ]:
# val_data = {
#     'image': image_file_paths,
#     'text': texts_csv_lst,
#     'label': transformed_genre_labels,
# }

In [ ]:
image_file_paths[0], texts_csv_lst[0], transformed_genre_labels[0]

In [ ]:
test_image_file_paths[0], test_texts_csv_lst[0], test_transformed_genre_labels[0]

In [18]:
import torch
from PIL import Image as pil

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, texts, labels, processor):
        self.images = images
        self.texts = texts
        self.labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.processor(pil.open(image).convert("RGB"), text,  max_length=32, add_special_tokens=True,  is_split_into_words=False, padding="max_length", truncation=True, return_tensors="pt")
        for k,v in encoding.items():
          encoding[k] = v.squeeze()

        # encoding["labels"] = label
        encoding["labels"] = torch.tensor(label)

        return encoding

In [19]:
from transformers import ViltProcessor

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-nlvr2")

dataset = CustomDataset(images=image_file_paths,
                     texts=texts_csv_lst,
                     labels=transformed_genre_labels,
                     processor=processor)

test_dataset = CustomDataset(images=test_image_file_paths,
                     texts=test_texts_csv_lst,
                     labels=test_transformed_genre_labels,
                     processor=processor)

In [ ]:
dataset[0]

In [ ]:
test_dataset[0]

Collate_fun and Data Loader

In [20]:
import torch

def collate_fn(batch):
  return {
      "input_ids": torch.stack([x["input_ids"] for x in batch]),
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      # "attention_mask": torch.stack([x["attention_mask"] for x in batch]),
      # "token_type_ids": torch.stack([x["token_type_ids"] for x in batch]),
      # "pixel_mask": torch.stack([x["pixel_mask"] for x in batch]),
      "labels": torch.stack([torch.tensor(x["labels"]) for x in batch]),
  }

In [21]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=12, shuffle=True)
test_dataloader = DataLoader(test_dataset, collate_fn=collate_fn, batch_size=12, shuffle=False)

In [ ]:
# We have to check which one of these collate_fun will work for our case !

In [ ]:
# from torch.utils.data import DataLoader

# def collate_fn(batch):
#   input_ids = [item['input_ids'] for item in batch]
#   pixel_values = [item['pixel_values'] for item in batch]
#   attention_mask = [item['attention_mask'] for item in batch]
#   token_type_ids = [item['token_type_ids'] for item in batch]
#   labels = [item['labels'] for item in batch]

#   # create padded pixel values and corresponding pixel mask
#   encoding = processor.image_processor.pad(pixel_values, return_tensors="pt")

#   # create new batch
#   batch = {}
#   batch['input_ids'] = torch.stack(input_ids)
#   batch['attention_mask'] = torch.stack(attention_mask)
#   batch['token_type_ids'] = torch.stack(token_type_ids)
#   batch['pixel_values'] = encoding['pixel_values']
#   batch['pixel_mask'] = encoding['pixel_mask']
#   batch['labels'] = torch.stack(labels)

#   return batch

# train_dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=4, shuffle=True)

Check Dimensions

In [ ]:
batch = next(iter(train_dataloader))
test_batch = next(iter(test_dataloader))

In [ ]:
for k,v in batch.items():
  print(k, v.shape)

In [ ]:
for k,v in test_batch.items():
  print(k, v.shape)

In [ ]:
# input_ids torch.Size([4, 40])
# attention_mask torch.Size([4, 40])
# token_type_ids torch.Size([4, 40])
# pixel_values torch.Size([4, 3, 448, 576])
# pixel_mask torch.Size([4, 448, 576])
# labels torch.Size([4, 3129])

In [ ]:
# processor.decode(batch["input_ids"][0])

In [ ]:
# batch["pixel_values"][0]

import Model and Processor

In [ ]:
# label2id = {label: encoded_id for label, encoded_id in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}
# id2label = {encoded_id: label for encoded_id, label in enumerate(label_encoder.classes_)}

In [ ]:
# label2id

In [22]:
valid_labels = label_encoder.classes_.tolist()

In [ ]:
# valid_labels

In [ ]:
from transformers import ViltForImagesAndTextClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViltForImagesAndTextClassification.from_pretrained("dandelin/vilt-b32-finetuned-nlvr2",
                                                                label2id = {label: str(i) for i, label in enumerate(valid_labels)},
                                                                id2label = {str(i): label for i, label in enumerate(valid_labels)},
                                                                num_images = 1,
                                                                ignore_mismatched_sizes=True)

model.to(device)

In [ ]:
model.config

Training Loop

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
checkpoint_dir = "/content/Model/Models-Train-35/"

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(50):
   print(f"Epoch: {epoch}")
   train_loss = 0
   for batch in tqdm(train_dataloader):
        # get the inputs;
        batch = {k:v.to(device) for k,v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        # print("Loss:", loss.item())
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

   print("Training Loss: ", train_loss)
   print(f"Saving Model at epoch: {epoch}")

   processor.save_pretrained(checkpoint_dir)
   model.save_pretrained(checkpoint_dir)

   model.eval()
   preds = []
   orignal = []
   for batch in tqdm(test_dataloader):
       batch = {k:v.to(device) for k,v in batch.items()}

       outputs = model(**batch)

       scores = outputs.logits
       preds_current = torch.argmax(scores, dim=1)

       preds += preds_current.cpu().numpy().tolist()
       orignal += batch["labels"].detach().cpu().numpy().tolist()

   accuracy = accuracy_score(orignal, preds)
   precision = precision_score(orignal, preds, average='macro')
   recall = recall_score(orignal, preds, average='macro')
   f1 = f1_score(orignal, preds, average='macro')

   print(f"Accuracy: {accuracy}")
   print(f"Precision: {precision}")
   print(f"Recall: {recall}")
   print(f"F1-Score: {f1}")


In [ ]:
# Based on which collate_fun we can use, we will decide which training loop can be used !

In [ ]:
# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# model.train()
# for epoch in range(50):  # loop over the dataset multiple times
#    print(f"Epoch: {epoch}")
#    for batch in train_dataloader:
#         # get the inputs;
#         pixel_values = batch["pixel_values"].to(device)
#         input_ids = batch["input_ids"].to(device)
#         label_ids = batch["labels"].to(device)

#         # batch = {k:v.to(device) for k,v in batch.items()}

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = model(input_ids=input_ids, pixel_values=pixel_values.unsqueeze(0), labels=label_ids)
#         # outputs = model(**batch)
#         loss = outputs.loss
#         print("Loss:", loss.item())
#         loss.backward()
#         optimizer.step()

In [ ]:
# We also need to save the best performing model
# and for that we need to validate model after each epoch

In [24]:
from transformers import ViltForImagesAndTextClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViltForImagesAndTextClassification.from_pretrained("/content/Model/Models-Train-35/")

model.to(device)

ViltForImagesAndTextClassification(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(3, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_f

In [25]:
import numpy as np
# correct, total, all_true = 0, 0, 0
preds = []
orignal = []

for batch in tqdm(test_dataloader):
    batch = {k:v.to(device) for k,v in batch.items()}
    # pixel_values = batch["pixel_values"].to(device)
    # input_ids = batch["input_ids"].to(device)
    # label_ids = batch["labels"].to(device)

    # outputs = model(input_ids=input_ids, pixel_values=pixel_values.unsqueeze(0))
    outputs = model(**batch)
    # logits = outputs.logits

    scores = outputs.logits
    preds_current = torch.argmax(scores, dim=1)
    # print(preds_current, batch["labels"])
    # correct += sum( batch["labels"] == preds_current)
    preds += preds_current.cpu().numpy().tolist()
    orignal += batch["labels"].detach().cpu().numpy().tolist()
    # total+=batch["pixel_values"].shape[0]
    # all_true += sum(batch["labels"])


    # loss = outputs.loss
    # print(logits)
    # idx = np.argmax(logits.detach).item()
    # print("Predicted answer:", model.config.id2label[idx])

  0%|          | 0/1823 [00:00<?, ?it/s]

<ipython-input-20-6185ed4ca984>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.stack([torch.tensor(x["labels"]) for x in batch]),


In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = accuracy_score(orignal, preds)
precision = precision_score(orignal, preds, average='macro')
recall = recall_score(orignal, preds, average='macro')
f1 = f1_score(orignal, preds, average='macro')

In [27]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

Accuracy: 0.9590744890026979
Precision: 0.935976297050587
Recall: 0.9399313059455687
F1-Score: 0.9367510465858238
